In [4]:
import pandas as pd #
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./data/Landslide_dataSet2.csv"
#device = "102"
#save_path = f"./data/dev{device}_prepared.csv"
save_path = f"data/dataSet2_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

def clean_and_format_timestamp(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('(ICT)', '').strip()
    try:
        dt = pd.to_datetime(val)
        return dt.isoformat()
    except Exception:
        return None  

df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])

df = df.drop(columns=['Unnamed: 15'], errors='ignore')
#df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
#df = df[df['lng'] > 15.0]
df = df[df['soil'] > 0]
df = df[df['rain'] >= 0]
df = df[df['humi'] > 0]
df = df[(df['geo'] >= 0) & (df['geo'] < 50)]
df = df[df['temp'] > 0]
df = df[(df['devID'] > 0) & (df['devID'] < 1000)]
#df = df[df['timestamp'] > '2025-09-11']

#df = df[df['devID'] == int(device)]
df

C:\Users\ahmad\AppData\Local\Temp\ipykernel_17520\2421482183.py:13: DtypeWarning: Columns (2,5,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,iso_risk,lstm_score,lstm_risk,confidence,rf_risk
0,2025-07-30 03:28:43.522,101,24.400000,0.0,35.753414,54.635536,17.443285,3.613000,8.638051,99.898964,2025-07-30 03:18:38.00,0.000000,1,-0.042206,NaN
1,2025-07-30 03:28:44.756,102,20.900000,0.0,36.623943,51.436863,2.512411,2.438000,8.637977,99.898933,2025-07-30 03:18:38.00,0.000000,1,-0.053683,NaN
2,2025-07-30 03:28:45.992,101,24.500000,0.0,35.700008,54.654610,11.029456,2.470000,8.638051,99.898964,2025-07-30 03:18:39.00,0.000000,1,-0.028987,NaN
3,2025-07-30 03:28:47.259,101,24.400000,0.0,35.673306,55.825741,8.358195,1.267000,8.638053,99.898964,2025-07-30 03:19:00.00,0.000000,1,-0.020144,NaN
4,2025-07-30 03:28:48.473,102,20.900000,0.0,36.591900,53.292744,2.527225,3.717000,8.637976,99.898933,2025-07-30 03:19:00.00,0.000000,1,-0.036699,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260984,2025-10-21 02:45:07.164,104,19.700001,0.0,32.900002,63.599998,16.403057,8.640429,99.911758,0.013693,0,0.150991,0,0.999999,0.0
260989,2025-10-21 02:45:59.409,103,20.100000,0.0,32.299999,62.500000,11.278135,8.640360,99.911819,0.030232,0,0.027337,0,0.999999,0.0
260993,2025-10-21 02:47:00.750,104,19.700001,0.0,32.750000,62.849998,10.985930,8.640429,99.911758,0.032027,0,0.174430,0,0.999999,0.0
260994,2025-10-21 02:47:11.154,103,20.100000,0.0,32.200001,64.099998,7.785539,8.640360,99.911819,0.037758,0,0.018421,0,0.999999,0.0


In [5]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,lstm_score,confidence,rf_risk
count,218539,218539.000000,218539.000000,218539.000000,218539.000000,218539.000000,218539.000000,218539.000000,218539.000000,218539.000000,218539.000000,172710.000000,57675.000000
mean,2025-08-30 02:44:08.452576256,102.362256,20.711787,0.055000,29.096003,77.387469,8.025695,10.976491,50.452890,-11.941113,0.123330,0.337990,0.004491
min,2025-07-30 03:28:43.522000,101.000000,0.802948,0.000000,6.550000,24.228929,0.000000,-1.000000,-1.000000,-86325.800490,-0.074746,-0.179042,0.000000
25%,2025-08-04 17:06:45.348999936,101.000000,18.600000,0.000000,26.400000,69.199997,1.531515,8.640389,8.638010,0.015292,0.000000,0.021674,0.000000
50%,2025-08-11 15:26:48.007000064,102.000000,20.700001,0.000000,28.062868,79.503929,5.123312,9.207000,8.638041,99.898911,0.000000,0.056681,0.000000
75%,2025-09-21 12:25:19.106999808,103.000000,22.100000,0.000000,31.900000,84.686279,14.021947,11.120000,99.911758,99.898964,0.034933,0.946974,0.000000
max,2025-10-21 02:47:52.495000,999.000000,64.699997,36.321999,39.766918,98.800003,49.999493,120624.540000,9954.666992,1760.843822,187.449081,1.000000,1.000000
std,NaN,3.521499,2.933958,0.824953,3.525790,12.077217,7.376230,262.810419,63.130688,2422.353807,2.607601,0.443465,0.066862


In [6]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng', 'iso_risk', 'iso_score', 'lstm_risk']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,iso_risk,lstm_score,lstm_risk,confidence,rf_risk
0,2025-07-30 03:28:43.522,101,24.400000,0.0,35.753414,54.635536,17.443285,3.613000,8.638051,99.898964,NaN,0.000000,1.0,-0.042206,NaN
1,2025-07-30 03:28:44.756,102,20.900000,0.0,36.623943,51.436863,2.512411,2.438000,8.637977,99.898933,NaN,0.000000,1.0,-0.053683,NaN
2,2025-07-30 03:28:45.992,101,24.500000,0.0,35.700008,54.654610,11.029456,2.470000,8.638051,99.898964,NaN,0.000000,1.0,-0.028987,NaN
3,2025-07-30 03:28:47.259,101,24.400000,0.0,35.673306,55.825741,8.358195,1.267000,8.638053,99.898964,NaN,0.000000,1.0,-0.020144,NaN
4,2025-07-30 03:28:48.473,102,20.900000,0.0,36.591900,53.292744,2.527225,3.717000,8.637976,99.898933,NaN,0.000000,1.0,-0.036699,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260984,2025-10-21 02:45:07.164,104,19.700001,0.0,32.900002,63.599998,16.403057,8.640429,99.911758,0.013693,0.0,0.150991,0.0,0.999999,0.0
260989,2025-10-21 02:45:59.409,103,20.100000,0.0,32.299999,62.500000,11.278135,8.640360,99.911819,0.030232,0.0,0.027337,0.0,0.999999,0.0
260993,2025-10-21 02:47:00.750,104,19.700001,0.0,32.750000,62.849998,10.985930,8.640429,99.911758,0.032027,0.0,0.174430,0.0,0.999999,0.0
260994,2025-10-21 02:47:11.154,103,20.100000,0.0,32.200001,64.099998,7.785539,8.640360,99.911819,0.037758,0.0,0.018421,0.0,0.999999,0.0


In [7]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.set_index('timestamp')
#df['hour'] = df.index.hourผ
#df['day_of_week'] = df.index.dayofweek
df = df.reset_index() 

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218539 entries, 0 to 218538
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   timestamp   218539 non-null  datetime64[ns]
 1   devID       218539 non-null  int64         
 2   soil        218539 non-null  float64       
 3   rain        218539 non-null  float64       
 4   temp        218539 non-null  float64       
 5   humi        218539 non-null  float64       
 6   geo         218539 non-null  float64       
 7   lat         218539 non-null  float64       
 8   lng         218539 non-null  float64       
 9   iso_score   218539 non-null  float64       
 10  iso_risk    103504 non-null  float64       
 11  lstm_score  218539 non-null  float64       
 12  lstm_risk   205479 non-null  float64       
 13  confidence  172710 non-null  float64       
 14  rf_risk     57675 non-null   float64       
dtypes: datetime64[ns](1), float64(13), int64(1)
memory 

In [9]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
iso_risk      115035
lstm_risk      13060
confidence     45829
rf_risk       160864
dtype: int64


In [10]:
#df = df.dropna(subset=['humi']).reset_index(drop=True)

In [11]:
nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
iso_risk      115035
lstm_risk      13060
confidence     45829
rf_risk       160864
dtype: int64


In [ ]:
#df = df.dropna()
#
#nan_counts = df.isna().sum()
#nan_columns = nan_counts[nan_counts > 0]
#print("df102")
#print("📌 คอลัมน์ที่มีค่า NaN:")
#print(nan_columns)

In [ ]:
#df[(df["iso_risk"]==2) & (df["lstm_risk"]==2)]

In [12]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,iso_risk,lstm_score,lstm_risk,confidence,rf_risk
count,218539,218539.000000,218539.000000,218539.000000,218539.000000,218539.000000,218539.000000,218539.000000,218539.000000,218539.000000,103504.000000,218539.000000,205479.000000,172710.000000,57675.000000
mean,2025-08-30 02:44:08.452576256,102.362256,20.711787,0.055000,29.096003,77.387469,8.025695,10.976491,50.452890,-11.941113,0.081765,0.123330,0.145475,0.337990,0.004491
min,2025-07-30 03:28:43.522000,101.000000,0.802948,0.000000,6.550000,24.228929,0.000000,-1.000000,-1.000000,-86325.800490,0.000000,-0.074746,0.000000,-0.179042,0.000000
25%,2025-08-04 17:06:45.348999936,101.000000,18.600000,0.000000,26.400000,69.199997,1.531515,8.640389,8.638010,0.015292,0.000000,0.000000,0.000000,0.021674,0.000000
50%,2025-08-11 15:26:48.007000064,102.000000,20.700001,0.000000,28.062868,79.503929,5.123312,9.207000,8.638041,99.898911,0.000000,0.000000,0.000000,0.056681,0.000000
75%,2025-09-21 12:25:19.106999808,103.000000,22.100000,0.000000,31.900000,84.686279,14.021947,11.120000,99.911758,99.898964,0.000000,0.034933,0.000000,0.946974,0.000000
max,2025-10-21 02:47:52.495000,999.000000,64.699997,36.321999,39.766918,98.800003,49.999493,120624.540000,9954.666992,1760.843822,2.000000,187.449081,2.000000,1.000000,1.000000
std,NaN,3.521499,2.933958,0.824953,3.525790,12.077217,7.376230,262.810419,63.130688,2422.353807,0.288805,2.607601,0.359632,0.443465,0.066862


In [13]:
df['devID'].value_counts()

devID
101    75626
103    54901
104    48599
102    39409
999        3
105        1
Name: count, dtype: int64

In [14]:

df.to_csv(f"{save_path}", index=False)

print("✅ บันทึกเรียบร้อยแล้ว")

✅ บันทึกเรียบร้อยแล้ว
